In [1]:
import numpy as np
import pandas as pd
import pandas.api.types
import sklearn.metrics


In [2]:
class ParticipantVisibleError(Exception):
    pass


def get_condition(full_location: str) -> str:
    # Given an input like spinal_canal_stenosis_l1_l2 extracts 'spinal'
    for injury_condition in ['spinal', 'foraminal', 'subarticular']:
        if injury_condition in full_location:
            return injury_condition
    raise ValueError(f'condition not found in {full_location}')


def score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        row_id_column_name: str,
        any_severe_scalar: float
    ) -> float:
    '''
    Pseudocode:
    1. Calculate the sample weighted log loss for each medical condition:
    2. Derive a new any_severe label.
    3. Calculate the sample weighted log loss for the new any_severe label.
    4. Return the average of all of the label group log losses as the final score, normalized for the number of columns in each group.
       This mitigates the impact of spinal stenosis having only half as many columns as the other two conditions.
    '''

    target_levels = ['normal_mild', 'moderate', 'severe']

    # Run basic QC checks on the inputs
    if not pandas.api.types.is_numeric_dtype(submission[target_levels].values):
        raise ParticipantVisibleError('All submission values must be numeric')

    if not np.isfinite(submission[target_levels].values).all():
        raise ParticipantVisibleError('All submission values must be finite')

    if solution[target_levels].min().min() < 0:
        raise ParticipantVisibleError('All labels must be at least zero')
    if submission[target_levels].min().min() < 0:
        raise ParticipantVisibleError('All predictions must be at least zero')

    solution['study_id'] = solution['row_id'].apply(lambda x: x.split('_')[0])
    solution['location'] = solution['row_id'].apply(lambda x: '_'.join(x.split('_')[1:]))
    solution['condition'] = solution['row_id'].apply(get_condition)

    del solution[row_id_column_name]
    del submission[row_id_column_name]
    assert sorted(submission.columns) == sorted(target_levels)

    submission['study_id'] = solution['study_id']
    submission['location'] = solution['location']
    submission['condition'] = solution['condition']

    # condition_losses = []
    # condition_weights = []
    # for condition in ['spinal', 'foraminal', 'subarticular']:
    #     condition_indices = solution.loc[solution['condition'] == condition].index.values
    #     condition_loss = sklearn.metrics.log_loss(
    #         y_true=solution.loc[condition_indices, target_levels].values,
    #         y_pred=submission.loc[condition_indices, target_levels].values,
    #         sample_weight=solution.loc[condition_indices, 'sample_weight'].values
    #     )
    #     condition_losses.append(condition_loss)
    #     condition_weights.append(1)

    condition_losses = {}
    condition_weights = {}
    for condition in ['spinal', 'foraminal', 'subarticular']:
        condition_indices = solution.loc[solution['condition'] == condition].index.values
        condition_loss = sklearn.metrics.log_loss(
            y_true=solution.loc[condition_indices, target_levels].values,
            y_pred=submission.loc[condition_indices, target_levels].values,
            sample_weight=solution.loc[condition_indices, 'sample_weight'].values
        )
        condition_losses[condition] = condition_loss
        condition_weights[condition] = 1

    any_severe_spinal_labels = pd.Series(solution.loc[solution['condition'] == 'spinal'].groupby('study_id')['severe'].max())
    any_severe_spinal_weights = pd.Series(solution.loc[solution['condition'] == 'spinal'].groupby('study_id')['sample_weight'].max())
    any_severe_spinal_predictions = pd.Series(submission.loc[submission['condition'] == 'spinal'].groupby('study_id')['severe'].max())
    any_severe_spinal_loss = sklearn.metrics.log_loss(
        y_true=any_severe_spinal_labels,
        y_pred=any_severe_spinal_predictions,
        sample_weight=any_severe_spinal_weights
    )
    condition_losses["any"]=any_severe_spinal_loss
    condition_weights["any"] = any_severe_scalar
    return np.average(np.asarray(list(condition_losses.values())), weights=np.asarray(list(condition_weights.values()))), condition_losses

In [3]:
submission = pd.read_csv("eval_rdnet_tiny_ax5ch_mask0.1.csv").sort_values("row_id", ascending=True).reset_index(drop=True).copy()


submission.head()

,row_id,normal_mild,moderate,severe
0,100206310_left_neural_foraminal_narrowing_l1_l2,0.683105,0.300781,0.016098
1,100206310_left_neural_foraminal_narrowing_l2_l3,0.402588,0.553223,0.044159
2,100206310_left_neural_foraminal_narrowing_l3_l4,0.148315,0.727539,0.124146
3,100206310_left_neural_foraminal_narrowing_l4_l5,0.072266,0.507324,0.420410
4,100206310_left_neural_foraminal_narrowing_l5_s1,0.269531,0.643066,0.087280


In [4]:
train_main = pd.read_csv("../../../input/rsna-2024-lumbar-spine-degenerative-classification/train.csv")

solution = train_main.melt(id_vars=["study_id"], var_name="full_label", value_name="severity")
solution["row_id"] = solution.apply(lambda row: str(row.study_id) + "_" + row.full_label, axis=1)
solution.severity = solution.severity.fillna("Normal/Mild")
solution.loc[solution.severity == "Normal/Mild", "normal_mild"] = 1
solution.loc[solution.severity == "Moderate", "moderate"] = 1
solution.loc[solution.severity == "Severe", "severe"] = 1

solution.loc[solution.severity == "Normal/Mild", "sample_weight"] = 1
solution.loc[solution.severity == "Moderate", "sample_weight"] = 2
solution.loc[solution.severity == "Severe", "sample_weight"] = 4

solution = solution[["study_id", "row_id", "normal_mild", "moderate", "severe", "sample_weight"]]
solution = solution.fillna(0)
solution = solution.sort_values(by=["row_id"])
solution = solution[solution["row_id"].isin(submission.row_id)].sort_values("row_id", ascending=True).reset_index(drop=True).copy()

solution.head()

,study_id,row_id,normal_mild,moderate,severe,sample_weight
0,100206310,100206310_left_neural_foraminal_narrowing_l1_l2,1.0,0.0,0.0,1.0
1,100206310,100206310_left_neural_foraminal_narrowing_l2_l3,0.0,1.0,0.0,2.0
2,100206310,100206310_left_neural_foraminal_narrowing_l3_l4,0.0,1.0,0.0,2.0
3,100206310,100206310_left_neural_foraminal_narrowing_l4_l5,0.0,0.0,1.0,4.0
4,100206310,100206310_left_neural_foraminal_narrowing_l5_s1,0.0,1.0,0.0,2.0


In [5]:
solution = solution[["row_id", "normal_mild", "moderate", "severe", "sample_weight"]]
solution.head()
solution = solution.sort_values("row_id", ascending=True).reset_index(drop=True).copy()

In [6]:
s = score(solution.copy(), submission.copy(), "row_id", 1)
f"Score {s}"

"Score (0.42509585398557415, {'spinal': 0.3007977998216565, 'foraminal': 0.5152693856954428, 'subarticular': 0.5926139075340882, 'any': 0.291702322891109})"

In [7]:
s = score(solution.sort_values("row_id").copy(), submission.sort_values("row_id").copy(), "row_id", 1)
f"Score {s}"

"Score (0.42509585398557415, {'spinal': 0.3007977998216565, 'foraminal': 0.5152693856954428, 'subarticular': 0.5926139075340882, 'any': 0.291702322891109})"

In [8]:
s = score(solution.sort_values("row_id", ascending=True).copy(), submission.sort_values("row_id", ascending=False).copy(), "row_id", 1)
f"Score {s}"

"Score (0.42509585398557415, {'spinal': 0.3007977998216565, 'foraminal': 0.5152693856954428, 'subarticular': 0.5926139075340882, 'any': 0.291702322891109})"

In [9]:
s = score(solution.sort_values("row_id", ascending=False).copy(), submission.sort_values("row_id", ascending=True).copy(), "row_id", 1)
f"Score {s}"

"Score (0.42509585398557415, {'spinal': 0.3007977998216565, 'foraminal': 0.5152693856954427, 'subarticular': 0.5926139075340884, 'any': 0.291702322891109})"

: 

In [10]:
from sklearn.model_selection import GroupKFold

train_df = pd.read_csv(
    "../../../input/rsna-2024-lumbar-spine-degenerative-classification/train.csv"
)
train_df["fold_id"] = -1
for i, (train_index, valid_index) in enumerate(
    GroupKFold(n_splits=5).split(
        train_df, np.arange(len(train_df)), train_df.study_id
    )
):
    train_df.loc[valid_index, "fold_id"] = i


solution = train_main.melt(id_vars=["study_id"], var_name="full_label", value_name="severity")
solution["row_id"] = solution.apply(lambda row: str(row.study_id) + "_" + row.full_label, axis=1)
solution.severity = solution.severity.fillna("Normal/Mild")
solution.loc[solution.severity == "Normal/Mild", "normal_mild"] = 1
solution.loc[solution.severity == "Moderate", "moderate"] = 1
solution.loc[solution.severity == "Severe", "severe"] = 1

solution.loc[solution.severity == "Normal/Mild", "sample_weight"] = 1
solution.loc[solution.severity == "Moderate", "sample_weight"] = 2
solution.loc[solution.severity == "Severe", "sample_weight"] = 4

solution = solution[["study_id", "row_id", "normal_mild", "moderate", "severe", "sample_weight"]]
solution = solution.fillna(0)
solution = solution.sort_values(by=["row_id"])
solution = solution[solution["row_id"].isin(submission.row_id)].sort_values("row_id", ascending=True).reset_index(drop=True).copy()

solution = solution.merge(train_df[["study_id", "fold_id"]], on="study_id")
solution.head()

,study_id,row_id,normal_mild,moderate,severe,sample_weight,fold_id
0,1004726367,1004726367_left_neural_foraminal_narrowing_l1_l2,1.0,0.0,0.0,1.0,0
1,1004726367,1004726367_left_neural_foraminal_narrowing_l2_l3,1.0,0.0,0.0,1.0,0
2,1004726367,1004726367_left_neural_foraminal_narrowing_l3_l4,1.0,0.0,0.0,1.0,0
3,1004726367,1004726367_left_neural_foraminal_narrowing_l4_l5,1.0,0.0,0.0,1.0,0
4,1004726367,1004726367_left_neural_foraminal_narrowing_l5_s1,1.0,0.0,0.0,1.0,0


In [11]:
for i in range(5):
    s = score(solution[solution.fold_id == i].copy(), submission[solution.fold_id == i].copy(), "row_id", 1)
    print(f"Score {s}")

Score (0.43863528103753635, {'spinal': 0.3039514215893382, 'foraminal': 0.5309460217058181, 'subarticular': 0.6292125137597322, 'any': 0.2904311670952572})


ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required.

In [ ]:
submission

In [ ]:
submission[submission.normal_mild == 1/3]

In [ ]:
train_coord_df = pd.read_csv("train_coord_df.csv")
mask = solution.study_id.isin(train_coord_df.study_id)
solution = solution[mask]
submission = submission[mask]

In [ ]:
s = score(solution.sort_values("row_id", ascending=False).copy(), submission.sort_values("row_id", ascending=True).copy(), "row_id", 1)
f"Score {s}"

In [ ]:
for i in range(5):
    s = score(solution[solution.fold_id == i].copy(), submission[solution.fold_id == i].copy(), "row_id", 1)
    print(f"Score {s}")

In [ ]:
import pandas as pd

axial_keypoints_df = pd.read_csv("axial_keypoints_df.csv")
